In [2]:
import time
from time import sleep
import json
from datetime import datetime
import boto3
import pandas as pd
import numpy as np

# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',1000)

In [14]:
df_item = pd.read_csv('./content.csv')
df_item.head()

,content_code,content_name,series_code,series_name,episode_index,item_code,item_name,item_type,genre,content_type,keywords,tags,valid_time,expire_time,update_time,actor_display,writer_display,region,language,issue_year,description,duration,ishd,center_content_code,center_content_name,center_series_code,center_series_name,content_provider,cp_name,online_no,oid,oid_name,pid,pid_name,t_create_time,category_name,site_key
0,Umai:PROG/4861680@BESTV.SMG.SMG,4K-地球脉动-7,NaN,NaN,NaN,vodpackage_hq_11817541,地球脉动,vodPackage,纪实收费,纪实,NaN,自然,2019-03-25 11:19:16,2029-03-25 11:19:16,2020-04-01 18:00:01,NaN,NaN,英国,NaN,0,从南极到北极，从赤道到寒带，从非洲草原到热带雨林，再从荒凉峰顶到深邃大海，难以数计的生物以极其绝美的身姿呈现在世人面前。,2924,3.0,Umai:PROG/4861680@BESTV.SMG.SMG,4K-地球脉动-7,NaN,NaN,NaN,bestv,NaN,opg.opgcontent.8dbbf86a7d9144ac81df5dbec10d920b.mediacontent,NaN,opg.program.74d14dffd0d448a2972c9862c0415118.mediacontent,NaN,2021-01-15 03:12:13,海外精选,IPTV_SHYD_SH
1,Umai:PROG/4861859@BESTV.SMG.SMG,4K-地球脉动-8,NaN,NaN,NaN,vodpackage_hq_11817541,地球脉动,vodPackage,纪实收费,纪实,NaN,自然,2019-03-25 11:19:16,2029-03-25 11:19:16,2020-04-01 19:10:02,NaN,NaN,英国,NaN,0,从南极到北极，从赤道到寒带，从非洲草原到热带雨林，再从荒凉峰顶到深邃大海，难以数计的生物以极其绝美的身姿呈现在世人面前。,2924,3.0,Umai:PROG/4861859@BESTV.SMG.SMG,4K-地球脉动-8,NaN,NaN,NaN,bestv,NaN,opg.opgcontent.8dbbf86a7d9144ac81df5dbec10d920b.mediacontent,NaN,opg.program.6fcd9e6f8aa44a97a8df23fd00cfc5d8.mediacontent,NaN,2021-01-15 03:12:13,海外精选,IPTV_SHYD_SH
2,Umai:PROG/3659711@BESTV.SMG.SMG,HD-Day9：悬挂·高级训练-悬挂系统训练-FitTime,NaN,NaN,NaN,vodpackage_hq_11516739,悬挂系统训练,vodPackage,其他;,时尚生活,NaN,其他;,2017-11-15 17:43:27,2027-11-15 17:43:27,2020-04-13 13:50:02,NaN,NaN,大陆,NaN,0,NaN,756,1.0,Umai:PROG/3659711@BESTV.SMG.SMG,HD-Day9：悬挂·高级训练-悬挂系统训练-FitTime,NaN,NaN,NaN,bestv,NaN,NaN,NaN,NaN,NaN,2021-01-15 03:12:13,FitTime,IPTV_SHYD_SH
3,10000005160320905896000000000000,《我要锻炼》刘九儒 马霄戎-第四期-德云社德云七队小园子湖广会馆站 2020-hs,NaN,NaN,NaN,vodpackage_SHYDSH_10016181,德云社德云七队小园子湖广会馆站 2020,vodPackage,NaN,综艺,NaN,NaN,2020-11-12 12:50:42,2099-12-31 00:00:00,2020-11-13 15:20:02,刘九儒,德云社,NaN,NaN,2020,《德云社德云七队小园子湖广会馆站 2020》由德云七队队长孟鹤堂携七队相声演员在北京湖广会馆为观众带来精彩的相声演出。,1800,1.0,NaN,NaN,NaN,NaN,huashu,huashu,NaN,NaN,NaN,NaN,NaN,2021-01-15 03:12:13,NaN,IPTV_SHYD_SH
4,10000006010000000000000001888006,熊梓淇老年妆跳女团舞-第七期-笑起来真好看-mg,NaN,NaN,NaN,vodpackage_SHYDSH_10014427,笑起来真好看,vodPackage,NaN,综艺,NaN,NaN,2020-07-27 00:00:00,2099-12-30 00:00:00,2020-10-16 10:50:03,沈腾 谢娜 李诞,刘柳,内地,NaN,2020,熊梓淇老年妆跳女团舞-第7期,5280,2.0,NaN,NaN,NaN,NaN,mangguo,PANDASHYDSH,NaN,NaN,NaN,NaN,NaN,2021-01-15 03:12:13,NaN,IPTV_SHYD_SH


In [15]:
def get_item_name(item_id):
    return df_item[df_item['item_code']==item_id].iloc[0]['item_name']
get_item_name('seriespackage_coll_12139575')

'装台'

In [16]:
%store -r

In [7]:
userpersonalization_solution_version_arn

'arn:aws-cn:personalize:cn-north-1:657975005475:solution/package-userpersonalization/ddf2b6cd'

In [8]:
userpersonalization_create_campaign_response = personalize.create_campaign(
    name = "bestv-item-userpersonalization",
    solutionVersionArn = userpersonalization_solution_version_arn,
    minProvisionedTPS = 1
)

userpersonalization_campaign_arn = userpersonalization_create_campaign_response['campaignArn']
print(json.dumps(userpersonalization_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws-cn:personalize:cn-north-1:657975005475:campaign/bestv-item-userpersonalization",
  "ResponseMetadata": {
    "RequestId": "c31e5451-75f6-4728-a87b-f8d8b14a91c1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 23 Mar 2021 02:07:37 GMT",
      "x-amzn-requestid": "c31e5451-75f6-4728-a87b-f8d8b14a91c1",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [9]:
sims_create_campaign_response = personalize.create_campaign(
    name = "bestv-item-SIMS",
    solutionVersionArn = sims_solution_version_arn,
    minProvisionedTPS = 1
)

sims_campaign_arn = sims_create_campaign_response['campaignArn']
print(json.dumps(sims_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws-cn:personalize:cn-north-1:657975005475:campaign/bestv-item-SIMS",
  "ResponseMetadata": {
    "RequestId": "f78015af-e9e2-4d3e-b868-7f1ff4e15d83",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 23 Mar 2021 02:07:41 GMT",
      "x-amzn-requestid": "f78015af-e9e2-4d3e-b868-7f1ff4e15d83",
      "content-length": "89",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [10]:
rerank_create_campaign_response = personalize.create_campaign(
    name = "bestv-item-rerank",
    solutionVersionArn = rerank_solution_version_arn,
    minProvisionedTPS = 1
)

rerank_campaign_arn = rerank_create_campaign_response['campaignArn']
print(json.dumps(rerank_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws-cn:personalize:cn-north-1:657975005475:campaign/bestv-item-rerank",
  "ResponseMetadata": {
    "RequestId": "9632421e-3130-4c2c-8850-31ae12f019e6",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 23 Mar 2021 02:07:45 GMT",
      "x-amzn-requestid": "9632421e-3130-4c2c-8850-31ae12f019e6",
      "content-length": "91",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [19]:
!aws personalize list-campaigns --region cn-north-1

{
    "campaigns": [
        {
            "name": "bestv-item-SIMS",
            "campaignArn": "arn:aws-cn:personalize:cn-north-1:657975005475:campaign/bestv-item-SIMS",
            "status": "ACTIVE",
            "creationDateTime": 1616465261.669,
            "lastUpdatedDateTime": 1616465670.576
        },
        {
            "name": "bestv-item-rerank",
            "campaignArn": "arn:aws-cn:personalize:cn-north-1:657975005475:campaign/bestv-item-rerank",
            "status": "ACTIVE",
            "creationDateTime": 1616465265.494,
            "lastUpdatedDateTime": 1616465791.548
        },
        {
            "name": "bestv-item-userpersonalization",
            "campaignArn": "arn:aws-cn:personalize:cn-north-1:657975005475:campaign/bestv-item-userpersonalization",
            "status": "ACTIVE",
            "creationDateTime": 1616465258.332,
            "lastUpdatedDateTime": 1616465761.9
        }
    ]
}


In [1]:
rec_hist_df= pd.read_csv('./rec_hist.csv')
rec_hist_df.head(3)

NameError: name 'pd' is not defined

In [41]:
#rec_hist_df["log_datetime"].max()
rec_hist_df[rec_hist_df["is_play"]==1].head(5)

,user_id,session_id,strategy_id,return_pos_id,return_item_id,log_datetime,is_play,rec_session_duration,is_click,is_exposure,year,month,day
1244,00400300000109006001C0132B44E0B9,e3765fe173bc4e78a7a30298c83e32ac,mix-1,1,seriespackage_hq_11299818,2020-11-19 15:07:45,1,10.0,1,1,2020,11,19
2373,00400300000100600001842C800F8D8C,f4946641184946e38bebd4d5bb291bcd,dnn-derulerank,12,seriespackage_hq_11057942,2020-12-29 16:37:19,1,0.0,1,0,2020,12,29
3392,00400300000109006001C0132B9DBF44,9a9a4af1a4054276a3f89bec8b3723b9,dianshimao-pre,0,seriespackage_cms_12051375,2020-11-09 19:38:39,1,2751.0,1,1,2020,11,9
3773,00400300000109006001C0132B9DBF44,764911728291491d931c75506521bb49,dianshimao-pre,3,seriespackage_cms_12051375,2020-11-08 21:00:48,1,153.0,1,1,2020,11,8
3776,00400300000109006001C0132B9DBF44,78d05c576dc94d28bbf846f4d6a30e9c,dianshimao-pre,5,seriespackage_cms_12051375,2020-11-15 17:13:35,1,9.0,1,1,2020,11,15


In [44]:
#rec_hist_df[rec_hist_df["user_id"]=='0040030000010060000100E400F9AED2']
item_list = rec_hist_df[(rec_hist_df["user_id"]=='00400300000100600001842C800F8D8C') & (rec_hist_df["session_id"]=='f4946641184946e38bebd4d5bb291bcd')]
item_list=pd.merge(item_list,df_item[["item_code","item_name"]].drop_duplicates(),left_on='return_item_id',right_on='item_code')
item_list.sort_values(by='return_pos_id', axis=0, ascending=True, inplace=True, kind='quicksort', na_position='last', ignore_index=False, key=None)
item_list[["item_code","item_name","return_pos_id","is_play","is_click"]]
#for item in item_list:
    #print('%s %s %' % (item['return_item_id'], get_item_name(item['return_item_id']), item['log_datetime'] ))
   # print(item["return_item_id"]+'|'+get_item_name(item['return_item_id'])+'|'+item["log_datetime"] )

,item_code,item_name,return_pos_id,is_play,is_click
5,seriespackage_cms_12048104,平行世界,0,0,0
6,seriespackage_cms_11869259,傲骨贤妻,1,0,0
8,seriespackage_hq_12139425,只为那一刻与你相见,2,0,0
4,seriespackage_cms_11958139,游戏幻影,3,0,0
7,seriespackage_cms_12056794,谁说我结不了婚,4,0,0
1,seriespackage_cms_12071089,皮影,5,0,0
0,seriespackage_cms_11949685,密战无声,6,0,0
14,seriespackage_cms_12132754,绝地反击,7,0,0
12,seriespackage_cms_12108160,锁定目标,8,0,0
13,seriespackage_cms_12108122,花女的抗战,9,0,0


Rerank:


In [46]:
rerank_resp = personalize_runtime.get_personalized_ranking(
    campaignArn = "arn:aws-cn:personalize:cn-north-1:657975005475:campaign/bestv-item-rerank",
    userId = "00400300000100600001842C800F8D8C",
    inputList = item_list["item_code"].tolist())

print("Personalized Ranking")
for item in rerank_resp['personalizedRanking']:
    print('%40s %s' % (item['itemId'], get_item_name(item['itemId'])))

Personalized Ranking
               seriespackage_hq_12139425 只为那一刻与你相见
              seriespackage_cms_12114131 走出国门
              seriespackage_cms_11869259 傲骨贤妻
              seriespackage_cms_12108122 花女的抗战
              seriespackage_cms_12132754 绝地反击
              seriespackage_cms_12109088 别逼我结婚
              seriespackage_cms_11949685 密战无声
              seriespackage_cms_12048104 平行世界
              seriespackage_cms_12108160 锁定目标
              seriespackage_cms_12131795 古灵精探B
              seriespackage_cms_11958139 游戏幻影
               seriespackage_hq_11057942 到爱的距离
              seriespackage_cms_12056794 谁说我结不了婚
              seriespackage_cms_12071089 皮影
               seriespackage_hq_11100310 热血


In [84]:
play_all_df=rec_hist_df[rec_hist_df["is_play"]==1].drop_duplicates(["user_id","session_id"])
dnn_hist_df=play_all_df[play_all_df["strategy_id"]=='dnn-derulerank']
dnn_hist_df.info()
strategies=play_all_df.groupby('strategy_id').agg({"session_id": np.count_nonzero}).reset_index()
strategies

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1138 entries, 2373 to 1696149
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   user_id               1138 non-null   object 
 1   session_id            1138 non-null   object 
 2   strategy_id           1138 non-null   object 
 3   return_pos_id         1138 non-null   int64  
 4   return_item_id        1138 non-null   object 
 5   log_datetime          1138 non-null   object 
 6   is_play               1138 non-null   int64  
 7   rec_session_duration  1138 non-null   float64
 8   is_click              1138 non-null   int64  
 9   is_exposure           1138 non-null   int64  
 10  year                  1138 non-null   int64  
 11  month                 1138 non-null   int64  
 12  day                   1138 non-null   int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 124.5+ KB


,strategy_id,session_id
0,dianshimao-pre,4033
1,dnn-derulerank,1138
2,mix-1,406
3,shichang,5731


In [95]:
#rec_hist_df[rec_hist_df["is_play"]==1].drop_duplicates(["user_id","session_id","strategy_id","return_item_id","return_pos_id","log_datetime"])
play_all_df=rec_hist_df[rec_hist_df["is_play"]==1].drop_duplicates(["user_id","session_id"])
dnn_hist_df=play_all_df[play_all_df["strategy_id"]=='dnn-derulerank']
recommend_df=pd.DataFrame(columns=['session_id','recommend_pos'])
#rec_hist_df[rec_hist_df["is_play"]==1].drop_duplicates(["user_id","session_id"]).head(10)
for row in dnn_hist_df.iterrows():
    item_list = rec_hist_df[(rec_hist_df["user_id"]==row[1]["user_id"]) & (rec_hist_df["session_id"]==row[1]["session_id"])]
    #item_list=pd.merge(item_list,df_item[["item_code","item_name"]].drop_duplicates(),left_on='return_item_id',right_on='item_code')
    item_list.sort_values(by='return_pos_id', axis=0, ascending=True, inplace=True)
    #item_list[["item_code","item_name","return_pos_id","is_play","is_click"]]
    rerank_resp = personalize_runtime.get_personalized_ranking(
                        campaignArn = "arn:aws-cn:personalize:cn-north-1:657975005475:campaign/bestv-item-rerank",
                        userId = row[1]["user_id"],
                        inputList = item_list["return_item_id"].tolist())
    
    
    #rerank_resp['personalizedRanking'][rerank_resp['personalizedRanking']["itemId"]==watch["item_code"]]
    #print('%40s %s' % (watch["session_id"],watch["item_code"],watch["return_pos_id"],item['itemId'], get_item_name(item['itemId'])))  
    i=-1  
    recommend_pos='-1'                                                                      
    for item in rerank_resp['personalizedRanking']:
        i=i+1
        if (item['itemId']==row[1]["return_item_id"]):
            recommend_pos=str(i)
            print('%40s %s' % (row[1]["session_id"], recommend_pos))
    
            break             
                                                                          
    #new_row = {'session_id':row[1]["session_id"], 'recommend_pos':recommend_pos}
    new_row=[[row[1]["session_id"],recommend_pos]]
    recommend_df.append(new_row, ignore_index=False)
    #print("ReRanking User Session: "+row[1]["session_id"])

dnn_hist_df=pd.merge(play_hist_df,recommend_df)    

dnn_hist_df.head(5)       

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


        f4946641184946e38bebd4d5bb291bcd 11
        3e2d68bcb0c240bba74c98f0c0e8ba49 13
        8ba0c9770cb640ffb5971eb99b354012 10
        7e75845a27cf4fa297b02a289dad822e 1
        46279eafbd2546d185a94ad9d8f079aa 11
        c283b0e9f03c43cf80c411727ec23dd9 4
        c72d6a0f2e534a47ba0581289ea04240 13
        1e28aaf57f414a569eaeb1c554383c78 9
        0f5ebdde0e01491196e3c7485c8085ac 5
        20d6c73b63ab4dcabf62936d23cf6d0e 10
        645bced20dd84a7fa55e4c70c9e2f8c5 5
        7dc314cf251744bcbb6e43d8fdfdfb7b 5
        8e7c62b513414d72abff95b55df44198 7
        a628b965ea634bbe9cc964f98821052d 13
        9cf4b1e3b70b4a9d9fa1fd34c940602d 10
        b3686d38e2514ed6a3034a73f5dd9329 5
        a35fc4fc12514385a42b3f45dbad3d43 10
        093ded690d4348b7b6a742b04396831b 14
        32a02e20d3e14a8188a3fc98b995bffc 4
        a34f13a90de3467dbdb7d4a703eb1575 8
        dcd4814f8aff42dfa5c851aab639d295 14
        46b3ba96f3ba4c4b80be62740ef42c31 10
        55145066405c427992d29779348c8273 1

,user_id,session_id,strategy_id,return_pos_id,return_item_id,log_datetime,is_play,rec_session_duration,is_click,is_exposure,year,month,day,recommend_pos


In [99]:
dnn_rerank_df=pd.read_csv('dnn-rerank.csv')
dnn_rerank_df.head(5)


,session_id,aws_position
0,f4946641184946e38bebd4d5bb291bcd,11
1,3e2d68bcb0c240bba74c98f0c0e8ba49,13
2,8ba0c9770cb640ffb5971eb99b354012,10
3,7e75845a27cf4fa297b02a289dad822e,1
4,46279eafbd2546d185a94ad9d8f079aa,11


In [101]:
play_all_df=rec_hist_df[rec_hist_df["is_play"]==1].drop_duplicates(["user_id","session_id"])
dnn_hist_df=play_all_df[play_all_df["strategy_id"]=='dnn-derulerank']
dnn_hist_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1138 entries, 2373 to 1696149
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   user_id               1138 non-null   object 
 1   session_id            1138 non-null   object 
 2   strategy_id           1138 non-null   object 
 3   return_pos_id         1138 non-null   int64  
 4   return_item_id        1138 non-null   object 
 5   log_datetime          1138 non-null   object 
 6   is_play               1138 non-null   int64  
 7   rec_session_duration  1138 non-null   float64
 8   is_click              1138 non-null   int64  
 9   is_exposure           1138 non-null   int64  
 10  year                  1138 non-null   int64  
 11  month                 1138 non-null   int64  
 12  day                   1138 non-null   int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 124.5+ KB


In [102]:
dnn_hist_df=pd.merge(dnn_hist_df,dnn_rerank_df)    

dnn_hist_df.head(5)      

,user_id,session_id,strategy_id,return_pos_id,return_item_id,log_datetime,is_play,rec_session_duration,is_click,is_exposure,year,month,day,aws_position
0,00400300000100600001842C800F8D8C,f4946641184946e38bebd4d5bb291bcd,dnn-derulerank,12,seriespackage_hq_11057942,2020-12-29 16:37:19,1,0.0,1,0,2020,12,29,11
1,21332420711382,3e2d68bcb0c240bba74c98f0c0e8ba49,dnn-derulerank,6,seriespackage_hq_11332706,2020-12-27 20:24:39,1,2734.0,1,1,2020,12,27,13
2,21332420711382,8ba0c9770cb640ffb5971eb99b354012,dnn-derulerank,5,seriespackage_hq_11832700,2020-12-16 20:49:02,1,20.0,1,1,2020,12,16,10
3,21332420711382,7e75845a27cf4fa297b02a289dad822e,dnn-derulerank,0,seriespackage_cms_12114033,2020-12-09 00:43:46,1,2670.0,1,1,2020,12,9,1
4,00400300000395500005748F1B399F66,46279eafbd2546d185a94ad9d8f079aa,dnn-derulerank,0,seriespackage_cms_12044319,2020-11-28 12:07:59,1,11.0,1,1,2020,11,28,11


In [111]:
dnn_hist_df["var"]=dnn_hist_df["aws_position"]-dnn_hist_df["return_pos_id"]
dnn_hist_df["var"].describe()

count    1138.000000
mean        1.987698
std         5.416986
min       -13.000000
25%        -2.000000
50%         2.000000
75%         6.000000
max        14.000000
Name: var, dtype: float64

In [112]:
dnn_hist_df[dnn_hist_df["var"]>6]

,user_id,session_id,strategy_id,return_pos_id,return_item_id,log_datetime,is_play,rec_session_duration,is_click,is_exposure,year,month,day,aws_position,var
1,21332420711382,3e2d68bcb0c240bba74c98f0c0e8ba49,dnn-derulerank,6,seriespackage_hq_11332706,2020-12-27 20:24:39,1,2734.0,1,1,2020,12,27,13,7
4,00400300000395500005748F1B399F66,46279eafbd2546d185a94ad9d8f079aa,dnn-derulerank,0,seriespackage_cms_12044319,2020-11-28 12:07:59,1,11.0,1,1,2020,11,28,11,11
6,00400300000395500005748F1B399F66,c72d6a0f2e534a47ba0581289ea04240,dnn-derulerank,6,seriespackage_hq_11810877,2020-12-21 12:08:24,1,252.0,1,1,2020,12,21,13,7
9,00400300000395500005748F1B399F66,20d6c73b63ab4dcabf62936d23cf6d0e,dnn-derulerank,2,seriespackage_cms_11984709,2020-11-24 10:11:45,1,18.0,1,1,2020,11,24,10,8
13,00400300000395500006748F1B3902F3,a628b965ea634bbe9cc964f98821052d,dnn-derulerank,0,Umai:SERI/5734839@BESTV.SMG.SMG,2020-11-25 21:18:13,1,2719.0,1,1,2020,11,25,13,13
16,21332351802394,a35fc4fc12514385a42b3f45dbad3d43,dnn-derulerank,3,seriespackage_cms_12109090,2020-12-29 12:29:02,1,2724.0,1,1,2020,12,29,10,7
17,21332351802394,093ded690d4348b7b6a742b04396831b,dnn-derulerank,4,seriespackage_hq_11567105,2020-12-29 21:14:31,1,142.0,1,1,2020,12,29,14,10
19,21332351802394,a34f13a90de3467dbdb7d4a703eb1575,dnn-derulerank,1,seriespackage_cms_12111949,2020-12-29 21:09:33,1,145.0,1,1,2020,12,29,8,7
20,21332351802394,dcd4814f8aff42dfa5c851aab639d295,dnn-derulerank,0,seriespackage_cms_12116204,2020-11-09 15:28:00,1,40.0,1,1,2020,11,9,14,14
26,00400300000395500008748F1B40D5C7,55b4ceb9ad684912aca5643dc48f91e0,dnn-derulerank,2,seriespackage_cms_11973761,2020-11-28 20:03:56,1,0.0,1,1,2020,11,28,12,10


In [74]:
#rec_hist_df[rec_hist_df["is_play"]==1].drop_duplicates(["user_id","session_id","strategy_id","return_item_id","return_pos_id","log_datetime"])
play_hist_df=rec_hist_df[rec_hist_df["is_play"]==1].drop_duplicates(["user_id","session_id"])
recommend_df=pd.DataFrame(columns=['session_id','recommend_pos'])
#rec_hist_df[rec_hist_df["is_play"]==1].drop_duplicates(["user_id","session_id"]).head(10)
for row in play_hist_df.iterrows():
    item_list = rec_hist_df[(rec_hist_df["user_id"]==row[1]["user_id"]) & (rec_hist_df["session_id"]==row[1]["session_id"])]
    #item_list=pd.merge(item_list,df_item[["item_code","item_name"]].drop_duplicates(),left_on='return_item_id',right_on='item_code')
    item_list.sort_values(by='return_pos_id', axis=0, ascending=True, inplace=True)
    #item_list[["item_code","item_name","return_pos_id","is_play","is_click"]]
    rerank_resp = personalize_runtime.get_personalized_ranking(
                        campaignArn = "arn:aws-cn:personalize:cn-north-1:657975005475:campaign/bestv-item-rerank",
                        userId = row[1]["user_id"],
                        inputList = item_list["return_item_id"].tolist())
    
    print("ReRanking User Session: "+row[1]["session_id"])
    #rerank_resp['personalizedRanking'][rerank_resp['personalizedRanking']["itemId"]==watch["item_code"]]
    #print('%40s %s' % (watch["session_id"],watch["item_code"],watch["return_pos_id"],item['itemId'], get_item_name(item['itemId'])))  
    i=-1  
    recommend_pos=-1                                                                      
    for item in rerank_resp['personalizedRanking']:
        i=i+1
        if (item['itemId']==row[1]["return_item_id"]):
            recommend_pos=i
            break                
                                                                          
    #new_row = {'session_id':row[1]["session_id"], 'recommend_pos':recommend_pos}
    new_row=[[row[1]["session_id"],recommend_pos]]
    recommend_df.append(new_row, ignore_index=False)

play_hist_df=pd.merge(play_hist_df,recommend_df)    

play_hist_df.head(5)                                                                         

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


ReRanking User Session: e3765fe173bc4e78a7a30298c83e32ac
ReRanking User Session: f4946641184946e38bebd4d5bb291bcd
ReRanking User Session: 9a9a4af1a4054276a3f89bec8b3723b9
ReRanking User Session: 764911728291491d931c75506521bb49
ReRanking User Session: 78d05c576dc94d28bbf846f4d6a30e9c
ReRanking User Session: fe15710216cd414bb06dd9c5dad91ac5
ReRanking User Session: 800fe9202bd94e84b7435ea2f0d4e4e8
ReRanking User Session: c625f994a9d74c7a8631384b8d2cd234
ReRanking User Session: 7a780ba76a9f494d831bc5ccc2a4ea35
ReRanking User Session: 4779ef286db84b24a2ac2ac76d02d363
ReRanking User Session: e073fd610bb341b59d9d9532ce2c8fc1
ReRanking User Session: 7e11a51b3c4242538cc0b8e56bb661d0
ReRanking User Session: 016fa1ae29f54db7a39f621fd81ef6f1
ReRanking User Session: 7ecee2418e03401ab556d7d1ab8eff7a
ReRanking User Session: 4d7e2a422ba649bda4068b7b9ac4daaf
ReRanking User Session: c68de2b2ccb34eb8a63d97d33e70aa28
ReRanking User Session: 77289e5c84444ad988f5585aeae3c325
ReRanking User Session: 4949592

KeyboardInterrupt: 

In [31]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = 'arn:aws-cn:personalize:cn-north-1:657975005475:campaign/bestv-item-userpersonalization',
    userId = str('0040030000010060000100E400F9AED2'),
    numResults = 16
)

print('%40s \n' % (get_recommendations_response['recommendationId']))
item_list = get_recommendations_response['itemList']
for item in item_list:
    print('%40s %s' % (item['itemId'], get_item_name(item['itemId'])))

RID-4aa60317-4914-40f6-a9ac-f3d043e14f4d 

               seriespackage_hq_11708289 喋血长江
              seriespackage_cms_12111659 第一茶庄
              seriespackage_cms_12113881 红蝎子
                  vodpackage_hq_12028018 中国维和行动
                      movie_cms_11952286 狗十三
               seriespackage_hq_12137314 不可能不爱你
                  vodpackage_hq_12058623 隐秘战士
              seriespackage_cms_12114033 冷枪
              seriespackage_cms_12129945 银楼金粉
                      movie_cms_12029364 1/2的魔法
                      movie_cms_12138094 猛禽小队和哈莉奎茵
                      movie_cms_12130721 信条
                   movie_SHYDSH_10018223 金刚川
              seriespackage_cms_12111254 天下有情之甜蜜蜜
               seriespackage_hq_12139425 只为那一刻与你相见
              seriespackage_cms_12112549 乐高幻影忍者第二季


In [ ]:
rerank_user = '0040030000010060000100E400F9AED2'
rerank_items = item_list
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

In [ ]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

In [ ]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df